In [1]:
import keras
from keras import layers
from keras.datasets import imdb
import numpy as np
nb_words = 10000
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=nb_words)

# get the word index and its reverse

In [2]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# get the actual review

In [3]:
def get_review_text(review):
   return " ".join([reverse_word_index.get(i - 3, "?") for i in review])

# print the actual review

In [4]:
print(get_review_text(train_data[6]))

? lavish production values and solid performances in this straightforward adaption of jane ? satirical classic about the marriage game within and between the classes in ? 18th century england northam and paltrow are a ? mixture as friends who must pass through ? and lies to discover that they love each other good humor is a ? virtue which goes a long way towards explaining the ? of the aged source material which has been toned down a bit in its harsh ? i liked the look of the film and how shots were set up and i thought it didn't rely too much on ? of head shots like most other films of the 80s and 90s do very good results


# define the one-hot encoding // The dimension is the number of words

In [5]:
def one_hot_encode(sequences,dimension):
    encodings = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
      for j in sequence:
          encodings[i,j] = 1
    return encodings

# train a binary classification model

In [6]:
def train_the_binary_classification(model,inputs,outputs,val_inputs,val_outputs):
  model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["binary_accuracy"])
  return model.fit(inputs,outputs,epochs=20,batch_size=512,validation_data=(val_inputs,val_outputs))

# Define the training and validation data

In [7]:
partial_train_data = train_data[10000:]
partial_train_labels = train_labels[10000:]
validation_data = train_data[:10000]
validation_labels = train_labels[:10000]

# Encode the data

In [9]:
# etrain_data = one_hot_encode(train_data,nb_words)
# etest_data = one_hot_encode(test_data,nb_words)
# evalidation_data = one_hot_encode(validation_data,nb_words)

train_encodings = one_hot_encode(partial_train_data, nb_words)
val_encodings = one_hot_encode(validation_data, nb_words)
test_encodings = one_hot_encode(test_data, nb_words)


# print the first encoding

In [11]:
# etrain_labels = np.asarray(train_labels).astype("float32")
# print(etrain_labels)
# etest_labels = np.asarray(test_labels).astype("float32")
# print(etest_labels)
# evalidation_labels = np.asarray(validation_labels).astype("float32")
# print(evalidation_labels)

print("First encoded training sample:", train_encodings[0])


First encoded training sample: [0. 1. 1. ... 0. 0. 0.]


# Define a good model; do not forget the last layer for binary classification

In [12]:
input = keras.Input(shape=(nb_words,),name="data")
l1 = layers.Dense(64,activation="relu")(input)
l2 = layers.Dense(64,activation="relu")(l1)
output = layers.Dense(1,activation="sigmoid")(l2) 
model = keras.Model(inputs=input,outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 data (InputLayer)           [(None, 10000)]           0         
                                                                 
 dense (Dense)               (None, 64)                640064    
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 644,289
Trainable params: 644,289
Non-trainable params: 0
_________________________________________________________________


# Train the binary classification with a validation set

In [13]:
# history = train_the_binary_classification(model,partial_train_data,partial_train_labels,validation_data,validation_labels)

history = train_the_binary_classification(model, train_encodings, partial_train_labels, val_encodings, validation_labels)


Epoch 1/20
30/30 [==============================] - 4s 91ms/step - loss: 0.4826 - binary_accuracy: 0.7759 - val_loss: 0.3233 - val_binary_accuracy: 0.8787
Epoch 2/20
30/30 [==============================] - 1s 29ms/step - loss: 0.2580 - binary_accuracy: 0.9052 - val_loss: 0.3239 - val_binary_accuracy: 0.8680
Epoch 3/20
30/30 [==============================] - 1s 32ms/step - loss: 0.1894 - binary_accuracy: 0.9289 - val_loss: 0.2772 - val_binary_accuracy: 0.8929
Epoch 4/20
30/30 [==============================] - 1s 23ms/step - loss: 0.1395 - binary_accuracy: 0.9506 - val_loss: 0.2947 - val_binary_accuracy: 0.8871
Epoch 5/20
30/30 [==============================] - 1s 29ms/step - loss: 0.1064 - binary_accuracy: 0.9641 - val_loss: 0.3192 - val_binary_accuracy: 0.8874
Epoch 6/20
30/30 [==============================] - 1s 22ms/step - loss: 0.0780 - binary_accuracy: 0.9755 - val_loss: 0.3511 - val_binary_accuracy: 0.8820
Epoch 7/20
30/30 [==============================] - 1s 21ms/step - los

# Evaluate on the training data

In [14]:
train_loss, train_acc = model.evaluate(train_encodings, partial_train_labels)
print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}")



469/469 [==============================] - 19s 41ms/step - loss: 9.0718e-05 - binary_accuracy: 1.0000
Training Loss: 0.0001, Training Accuracy: 1.0000


# Evaluate on the Validation Data

In [15]:
val_loss, val_acc = model.evaluate(val_encodings, validation_labels)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")



313/313 [==============================] - 12s 37ms/step - loss: 0.9014 - binary_accuracy: 0.8688
Validation Loss: 0.9014, Validation Accuracy: 0.8688


# Evaluate on the testing data

In [16]:
test_loss, test_acc = model.evaluate(test_encodings, test_labels)
print(f"Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_acc:.4f}")


782/782 [==============================] - 32s 40ms/step - loss: 0.9781 - binary_accuracy: 0.8546
Testing Loss: 0.9781, Testing Accuracy: 0.8546
